In [1]:
import pandas as pd, re, csv

In [2]:
df = pd.read_csv('netflix.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [3]:
df.columns = df.columns.str.title()
category = df.select_dtypes(include = 'object').columns
df[category] = df[category].apply(lambda x: x.str.strip().str.title())

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Show_Id       8807 non-null   object
 1   Type          8807 non-null   object
 2   Title         8807 non-null   object
 3   Director      6173 non-null   object
 4   Cast          7982 non-null   object
 5   Country       7976 non-null   object
 6   Date_Added    8797 non-null   object
 7   Release_Year  8807 non-null   int64 
 8   Rating        8803 non-null   object
 9   Duration      8804 non-null   object
 10  Listed_In     8807 non-null   object
 11  Description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [5]:
df.isna().sum()

Show_Id            0
Type               0
Title              0
Director        2634
Cast             825
Country          831
Date_Added        10
Release_Year       0
Rating             4
Duration           3
Listed_In          0
Description        0
dtype: int64

In [6]:
df['Has_Director'] = df['Director'].notna()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Show_Id       8807 non-null   object
 1   Type          8807 non-null   object
 2   Title         8807 non-null   object
 3   Director      6173 non-null   object
 4   Cast          7982 non-null   object
 5   Country       7976 non-null   object
 6   Date_Added    8797 non-null   object
 7   Release_Year  8807 non-null   int64 
 8   Rating        8803 non-null   object
 9   Duration      8804 non-null   object
 10  Listed_In     8807 non-null   object
 11  Description   8807 non-null   object
 12  Has_Director  8807 non-null   bool  
dtypes: bool(1), int64(1), object(11)
memory usage: 834.4+ KB


In [7]:
df['Director'] = df['Director'].fillna('Unknown')
df['Cast'] = df['Cast'].fillna('Unknown')
df['Country'] = df['Country'].fillna('Unknown')
df.isna().sum()

Show_Id          0
Type             0
Title            0
Director         0
Cast             0
Country          0
Date_Added      10
Release_Year     0
Rating           4
Duration         3
Listed_In        0
Description      0
Has_Director     0
dtype: int64

In [8]:
df['Date_Added'] = pd.to_datetime(df['Date_Added'], format = 'mixed')

In [9]:
df['Date_Added'] = df['Date_Added'].fillna(df['Date_Added'].mode()[0])
df['Rating'] = df['Rating'].fillna(df['Rating'].mode()[0])
df.isna().sum()

Show_Id         0
Type            0
Title           0
Director        0
Cast            0
Country         0
Date_Added      0
Release_Year    0
Rating          0
Duration        3
Listed_In       0
Description     0
Has_Director    0
dtype: int64

In [10]:
df['Duration'] = df['Duration'].fillna(df.groupby('Type')['Duration'].transform(lambda x: x.mode()[0]))
df.isna().sum()

Show_Id         0
Type            0
Title           0
Director        0
Cast            0
Country         0
Date_Added      0
Release_Year    0
Rating          0
Duration        0
Listed_In       0
Description     0
Has_Director    0
dtype: int64

In [11]:
df['Title'].duplicated().sum()

6

In [12]:
print(f"Duplicate Title\n{df['Title'][df['Title'].duplicated()]}")

Duplicate Title
5095         Fullmetal Alchemist
5751                  Death Note
6529                Consequences
6705        Esperando La Carroza
7345              Love In A Puff
8022    Sin Senos Sí Hay Paraíso
Name: Title, dtype: object


In [13]:
df = df.drop_duplicates(subset = 'Title', keep = 'first').reset_index(drop = True)

In [14]:
df['Title'].duplicated().sum()

0

In [15]:
df['Added_Year'] = df['Date_Added'].dt.year
df['Added_Month'] = df['Date_Added'].dt.month_name()

In [16]:
df['Listed_In'].value_counts()

Listed_In
Dramas, International Movies                          362
Documentaries                                         359
Stand-Up Comedy                                       334
Comedies, Dramas, International Movies                273
Dramas, Independent Movies, International Movies      252
                                                     ... 
Kids' Tv, Tv Action & Adventure, Tv Dramas              1
Tv Comedies, Tv Dramas, Tv Horror                       1
Children & Family Movies, Comedies, Lgbtq Movies        1
Kids' Tv, Spanish-Language Tv Shows, Teen Tv Shows      1
Cult Movies, Dramas, Thrillers                          1
Name: count, Length: 514, dtype: int64

In [17]:
df['Listed_In'] = df['Listed_In'].apply(lambda x: [c.strip() for c in re.split(r',|&', x) if c.strip() != ''])
df_exploded1 = df.explode('Listed_In')

In [18]:
df_exploded1['Listed_In'].value_counts()

Listed_In
International Movies         2749
Dramas                       2425
Comedies                     1672
International Tv Shows       1348
Adventure                    1027
Documentaries                 869
Action                        859
Tv Dramas                     762
Independent Movies            756
Children                      641
Family Movies                 641
Romantic Movies               616
Tv Comedies                   581
Thrillers                     576
Crime Tv Shows                469
Kids' Tv                      451
Stand-Up Comedy               399
Docuseries                    395
Musicals                      375
Music                         375
Romantic Tv Shows             370
Horror Movies                 357
Fantasy                       327
Reality Tv                    255
British Tv Shows              253
Sci-Fi                        243
Sports Movies                 219
Anime Series                  174
Spanish-Language Tv Shows     173
Tv A

In [19]:
genre = ['Drama', 'Comedy', 'International', 'Crime', 'Horror', 'British', 'Documentary', 'Anime', 'Reality Show', 'Classic', 'Action', 'Romance', 'Independent', 'Sci-Fi', 'Sports', 'Thriller']
df_exploded1['Listed_In'] = df_exploded1['Listed_In'].replace({'Dramas' : genre[0], 'Comedies' : genre[1], 'Documentaries' : genre[6], 'International Tv Shows' : genre[2],
                                      'Crime Tv Shows' : genre[3], "Kids' Tv" : 'Children', 'Stand-Up Comedy' : genre[1], 'Horror Movies' : genre[4],
                                      'British Tv Shows' : genre[5], 'Docuseries' : genre[6], 'Anime Series' : genre[7], 'International Movies' : genre[2],
                                      'Tv Comedies' : genre[1], 'Reality Tv' : genre[8], 'Classic Movies' : genre[9], 'Tv Dramas' : genre[0], 'Tv Action' : genre[10],
                                      'Romantic Tv Shows' : genre[11], 'Anime Features' : genre[7], 'Independent Movies' : genre[12], 'Cult Movies' : 'Cult',
                                       'Tv Horror' : genre[4], 'Romantic Movies' : genre[11], 'Spanish-Language Tv Shows' : 'Spanish', 'Lgbtq Movies' : 'LGBTQ',
                                       'Tv Sci-Fi' : genre[13], 'Sports Movies' : genre[14], 'Thrillers' : genre[15], 'Family Movies' : 'Family', 'Musicals' : 'Music',
                                        'Korean Tv Shows' : 'Korean', 'Tv Mysteries' : 'Mystery', 
                                        'Nature Tv' : 'Nature', 'Teen Tv Shows' : 'Teen', 'Spirituality' : 'Faith', 'Tv Thrillers' : genre[15], 'Cult Tv' : 'Cult'})

In [20]:
df_exploded1['Listed_In'].value_counts()

Listed_In
International    4097
Drama            3187
Comedy           2652
Documentary      1264
Children         1092
Action           1027
Adventure        1027
Romance           986
Independent       756
Music             750
Family            641
Thriller          633
Crime             469
Horror            432
Fantasy           327
Sci-Fi            327
Reality Show      255
British           253
Anime             245
Sports            219
Spanish           173
Korean            151
Classic           144
Faith             130
LGBTQ             102
Mystery            98
Cult               98
Science            92
Nature             92
Teen               69
Movies             57
Talk Shows         56
Tv Shows           16
Name: count, dtype: int64

In [21]:
df['Country'].value_counts()

Country
United States                             2818
India                                      972
Unknown                                    831
United Kingdom                             419
Japan                                      243
                                          ... 
Romania, Bulgaria, Hungary                   1
Uruguay, Guatemala                           1
France, Senegal, Belgium                     1
Mexico, United States, Spain, Colombia       1
United Arab Emirates, Jordan                 1
Name: count, Length: 749, dtype: int64

In [22]:
df['Country'] = df['Country'].apply(lambda x: [c.strip() for c in x.split(',') if c.strip() != ''])
df_exploded2 = df.explode('Country')

In [23]:
df_exploded2['Country'].value_counts()

Country
United States     3689
India             1046
Unknown            831
United Kingdom     806
Canada             445
                  ... 
Bermuda              1
Ecuador              1
Armenia              1
Mongolia             1
Montenegro           1
Name: count, Length: 123, dtype: int64

In [24]:
df_exploded1.to_csv('genre.csv', index = False, encoding = 'utf-8')
df_exploded2.to_csv('country.csv', index = False, encoding = 'utf-8')

In [25]:
df['Rating'].value_counts()

Rating
Tv-Ma       3208
Tv-14       2158
Tv-Pg        863
R            799
Pg-13        490
Tv-Y7        334
Tv-Y         307
Pg           287
Tv-G         220
Nr            79
G             41
Tv-Y7-Fv       6
Nc-17          3
Ur             3
74 Min         1
84 Min         1
66 Min         1
Name: count, dtype: int64

In [26]:
bad_rating = ['74 Min', '84 Min', '66 Min']
df = df[~df['Rating'].isin(bad_rating)]

In [27]:
df['Rating'] = df['Rating'].str.upper()
df['Rating'].value_counts()

Rating
TV-MA       3208
TV-14       2158
TV-PG        863
R            799
PG-13        490
TV-Y7        334
TV-Y         307
PG           287
TV-G         220
NR            79
G             41
TV-Y7-FV       6
NC-17          3
UR             3
Name: count, dtype: int64

In [53]:
df.to_csv('clean_df.csv', index = False, encoding = 'utf-8')

In [52]:
(df.applymap(lambda x: '\n' in str(x) if isinstance(x, str) else False)
   .sum())

C:\Users\nanda\AppData\Local\Temp\ipykernel_20128\261311734.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  (df.applymap(lambda x: '\n' in str(x) if isinstance(x, str) else False)


Show_Id         0
Type            0
Title           0
Director        0
Cast            0
Country         0
Date_Added      0
Release_Year    0
Rating          0
Duration        0
Listed_In       0
Description     0
Has_Director    0
Added_Year      0
Added_Month     0
dtype: int64

In [51]:
df = df.applymap(lambda x: str(x).replace('\n', ' ') if isinstance(x, str) else x)

C:\Users\nanda\AppData\Local\Temp\ipykernel_20128\1486372571.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).replace('\n', ' ') if isinstance(x, str) else x)
